# Quick analysis

In [1]:
## Import libraries

%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from datetime import date, datetime
from IPython.display import display, Markdown


In [13]:
# import data

df = pd.read_feather("../output/input.feather")
#df = pd.read_csv("../output/input.csv")

# for binary fields, replace zeros with null so that `count` can
binary_fields = ['elective_short_stay', 'elective_or_op']
for f in binary_fields:
    df[f] = df[f].replace(0,np.nan)

maxdate = df["any_admission_date"].max()
display(maxdate)
display(df["outpatient_covid_therapeutic_name"].unique())

'2022-04-25'

array(['paxlovid', 'molnupiravir', 'sotrovimab', nan, 'remdesivir',
       'casirivimab and imdevimab'], dtype=object)

In [14]:

fields = {
'Outpatient':
    ['outpatient_covid_therapeutic_date',
     'elective_or_op',
     'elective_short_stay', 
     'daycase_admission_date',
     'elective_x892_date', 
     'elective_x292_date', 
     'hospital_attendance_date'],
'Inpatient':      
    ['inpatient_covid_therapeutic_date',
     'any_admission_date', 
     'any_admission_x892_date',
     'any_admission_x292_date']
}

for x in fields:
    f = fields[x][0]
    display(Markdown(f"## {x}"))

    # filter to treatment dates within available SUS data range
    df1 = df.copy().loc[(df[f] <= maxdate)]

    if x=="Inpatient": # don't count admissions if discharge date was after treatment date
        display(Markdown(f"Note: for inpatients, recent spells may not yet have completed so some data may be missing"))
        for c in fields[x][1:]: # for each admission type
            # compare discharge date with treatment date and remove admission date if not in window
            df1.loc[df1[f] > df1[c.replace("admission", "discharge")], c] = np.nan
            
    # create df for mabs and separate df for Antivirals
    name_field = f'{x.lower()}_covid_therapeutic_name'
    df1[name_field] = df1[name_field].astype(str).fillna("")
    df_mab = df1.loc[df1[name_field].str.contains("mab")]
    df_non_mab = df1.loc[~df1[name_field].str.contains("mab")]
    treatments = {"MABs":df_mab,
                  "Antivirals":df_non_mab}

    # Breakdown by treatment type (MABs/Avs)
    for t in treatments:
        df_t = treatments[t]
        summary1 = pd.DataFrame(df_t[fields[x]].count()).rename(columns={0:"count"})
        summary1["percent"] = (100*(summary1["count"]/summary1["count"][f])).fillna(0).round(2).astype(int)
        summary1.index = summary1.index.str.replace("_date","")
        display(Markdown(f"### {t}"), summary1)

    # Breakdown by region and age group (MABs only, most inclusive hospital record field only)
    for col in ["region_covid_therapeutics","age_group"]:
        summary2 = pd.DataFrame(df_mab.groupby(col)\
                [fields[x][0:2]].count()).rename(columns={0:"count"})
        summary2["percent"] = (100*summary2[fields[x][1]]/summary2[f]).fillna(0).round(2).astype(int)
        summary2.columns = summary2.columns.str.replace("_date","")
        display(Markdown("### Breakdown by region and age group (MABs only)"), summary2)

## Outpatient

### MABs

,count,percent
outpatient_covid_therapeutic,278,100
elective_or_op,140,50
elective_short_stay,203,73
daycase_admission,148,53
elective_x892,29,10
elective_x292,26,9
hospital_attendance,37,13


### Antivirals

,count,percent
outpatient_covid_therapeutic,523,100
elective_or_op,268,51
elective_short_stay,360,68
daycase_admission,256,48
elective_x892,49,9
elective_x292,49,9
hospital_attendance,47,8


...

,count,percent
inpatient_covid_therapeutic,519,100
any_admission,186,35
any_admission_x892,136,26
any_admission_x292,123,23


### Breakdown by region and age group (MABs only)

,inpatient_covid_therapeutic,any_admission,percent
region_covid_therapeutics,,,
East,28,5,17
East Midlands,28,10,35
London,54,21,38
North East,31,12,38
North West,30,12,40
South East,38,13,34
South West,25,8,32
West Midlands,27,11,40
Yorkshire and The Humber,21,9,42


### Breakdown by region and age group (MABs only)

,inpatient_covid_therapeutic,any_admission,percent
age_group,,,
12-24,33,13,39
25-34,21,5,23
35-44,54,17,31
45-54,45,13,28
55-64,34,11,32
65-74,35,14,40
75+,31,12,38
missing,29,16,55
